In [1]:
import mosek.fusion as mf
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import yfinance as yf
import altair as alt
alt.renderers.set_embed_options(actions=False, theme="dark")

RendererRegistry.enable('default')

In [2]:
def build_model(
    n_assets: int,
    asset_sectors:pd.DataFrame,
    constraints:List[Dict[str, Any]],
)->mf.Model:
    """
    Function for building MOSEK model to solve the mean-variance optimization
    problem with diversification constraints.

    Parameters
    ----------
    n_assets : int
        Number of assets in the investment universe.
    asset_sectors : pd.DataFrame
        DataFrame containing assets' names (`"Asset"` column) and their 
        corresponding sectors (`"Sector"` column).
    constraints : List[Dict[str, Any]]
        List of diversification constraints (dictionaries). The dictionaries
        must have the following keys:

        - `"Type"`: type of constraint. It can be `"All Assets"` (the constraint
        applies to all the assets), `"Sectors"` (the constraint applies only
        to assets from a particular sector) or `"Assets"` (the constraint 
        applies to a particular asset).
        - `"Weight"`: limit value for the assets' weights.
        - `"Sign"`: domain of the constraint. It can be `">="` (greater than) or
        `"<="` (less than).
        - `"Position"`: indicates to which positions the constraint applies. It
        can be the name of a sector, the name of an asset or an empty string 
        (`""`) if the constraint type is `"All Assets"`.

    Returns
    -------
    model: mf.Model
        MOSEK model object.
    """
    # Creating the model
    model = mf.Model("markowitz_MV")

    # Weights variable with no-short-selling constraint.
    x = model.variable("x", n_assets, mf.Domain.greaterThan(0.0))

    # Variable for modeling the portfolio variance in the objective function
    s = model.variable("s", 1, mf.Domain.unbounded())

    # Parameter for cov matrix decomposition.
    G = model.parameter("G", [n_assets, n_assets])

    # Parameter for expected returns vector.
    mu = model.parameter("mu", n_assets)

    # delta parameter
    delta = model.parameter("delta")

    # Budget constraint (fully invested)
    model.constraint("budget", mf.Expr.sum(x), mf.Domain.equalsTo(1))

    # Iterate over the constraints list and add the constraints to the model.
    for c, constraint in enumerate(constraints):

        sign = (
            mf.Domain.greaterThan(constraint["Weight"]) 
            if constraint["Sign"] == ">=" else 
            mf.Domain.lessThan(constraint["Weight"])
        )

        if constraint["Type"] == "All Assets":

            A = np.identity(n_assets)

            model.constraint(
                f"c_{c}", 
                mf.Expr.mul(
                    A,
                    x
                ), 
                sign
            )
        
        elif constraint["Type"] == "Sectors":

            A = np.where(
                asset_sectors.loc[:, "Sector"] == constraint["Position"], 
                1., 
                0.
            )

            model.constraint(
                f"c_{c}",
                mf.Expr.dot(A, x),
                sign
            )
        
        elif constraint["Type"] == "Assets":

            A = np.where(
                asset_sectors.loc[:, "Asset"] == constraint["Position"], 
                1., 
                0.
            )

            model.constraint(
                f"c_{c}",
                mf.Expr.dot(A, x),
                sign
            )
        
    # Conic constraint for the portfolio variance
    model.constraint(
        "risk", 
        mf.Expr.vstack(
            s, 
            mf.Expr.mul(G, x)
        ), 
        mf.Domain.inQCone()
    )
    
    # Define objective function
    model.objective(
        "obj", 
        mf.ObjectiveSense.Maximize, 
        mf.Expr.sub(
            mf.Expr.dot(mu, x), 
            mf.Expr.mul(delta, s)
        )
    )

    return model


In [3]:
asset_sectors = pd.DataFrame(
    {
        "Asset": [
            "NVDA", 
            "AMD", 
            "INTC", 
            "BAC",
            "JPM",
            "C",
            "MSFT",
            "GOOG",
            "META",
            "BTC-USD",
            "ETH-USD",
        ],
        "Sector": [
            "Electronic Technology", 
            "Electronic Technology", 
            "Electronic Technology",
            "Finance",
            "Finance",
            "Finance",
            "Technology Services",
            "Technology Services",
            "Technology Services",
            "Crypto",
            "Crypto"
        ]
    }
)

assets_data = yf.download(asset_sectors.loc[:, "Asset"].to_list())

[*********************100%***********************]  11 of 11 completed


In [4]:
assets_returns = (
    assets_data.loc[:, "Adj Close"]
    .pct_change()
    .loc[:, asset_sectors.loc[:, "Asset"]]
)

sigma = assets_returns.cov()

mu = assets_returns.mean()

G = pd.DataFrame(
    np.linalg.cholesky(sigma), index=sigma.index, columns=sigma.columns
)


In [5]:
constraints = [
    {"Type": "All Assets", "Weight": 0.2, "Sign": "<=", "Position": ""},
    {
        "Type": "Sectors",
        "Weight": 0.3,
        "Sign": ">=",
        "Position": "Electronic Technology",
    },
    {
        "Type": "Sectors",
        "Weight": 0.4,
        "Sign": "<=",
        "Position": "Electronic Technology",
    },
    {
        "Type": "Assets",
        "Weight": 0.05,
        "Sign": "<=",
        "Position": "META",
    },
    {
        "Type": "Sectors",
        "Weight": 0.1,
        "Sign": "<=",
        "Position": "Crypto",
    },
]


In [6]:
n_assets = len(asset_sectors)

# Build constrained model
constrained_model = build_model(n_assets, asset_sectors, constraints)

# Set required parameters.

constrained_model.getParameter("G").setValue(G.to_numpy().T) # Remember to transpose G.

constrained_model.getParameter("mu").setValue(mu.to_numpy())

constrained_model.getParameter("delta").setValue(0.1)

# Solve optimization problem.
constrained_model.solve()

# Get optimal weights from the Model object.
weights = pd.Series(
    constrained_model.getVariable("x").level(),
    index=asset_sectors.loc[:, "Asset"],
    name="Constrained"
).to_frame()

# Build unconstrained model
unconstrained_model = build_model(n_assets, asset_sectors, [])

# Set required parameters.

unconstrained_model.getParameter("G").setValue(G.to_numpy().T) # Remember to transpose G.

unconstrained_model.getParameter("mu").setValue(mu.to_numpy())

unconstrained_model.getParameter("delta").setValue(0.1)

# Solve optimization problem.
unconstrained_model.solve()

# Get optimal weights from the Model object.

weights.loc[:, "Unconstrained"] = unconstrained_model.getVariable("x").level()

weights.style.format("{:.2%}")


,Constrained,Unconstrained
Asset,,
NVDA,11.34%,5.46%
AMD,1.89%,0.00%
INTC,16.77%,0.06%
BAC,0.52%,0.00%
JPM,14.48%,6.12%
C,0.00%,0.00%
MSFT,20.00%,25.97%
GOOG,20.00%,29.99%
META,5.00%,10.19%


In [7]:
weights = weights.stack().reset_index().rename({"level_1": "Optimization Type", 0: "Weight"}, axis=1)

In [8]:
c = (
    alt.Chart(weights)
    .mark_bar()
    .encode(
        x=alt.X("Optimization Type:N", axis=None),
        y=alt.Y("Weight:Q", axis=alt.Axis(format=".2%", title="Weight")),
        color=alt.Color(
            "Optimization Type:N",
            legend=alt.Legend(orient="bottom", title=None),
            scale=alt.Scale(range=["#e60049", "#0bb4ff"])
        ),
        column=alt.Column("Asset:N", spacing=5),
    )
)

c


/Users/matiasmacazaga/python/portfolio_optimization_with_mosek/.venv/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [111]:
c.save("weights_comparison.html")

In [9]:
deltas = np.linspace(0.01, 5, 100)

efficient_frontier = []

for d in deltas:

    constrained_model.getParameter("delta").setValue(d)

    constrained_model.solve()

    w = constrained_model.getVariable("x").level()

    portf_exp_return = np.dot(mu, w) * 252

    portf_volatility = constrained_model.getVariable("s").level()[0] * np.sqrt(252)

    efficient_frontier.append([portf_volatility, portf_exp_return, *list(w)])

efficient_frontier = pd.DataFrame(
    efficient_frontier,
    columns=[
        "volatility",
        "expected_return",
        *asset_sectors.loc[:, "Asset"].to_list(),
    ],
)

efficient_frontier.head()

,volatility,expected_return,NVDA,AMD,INTC,BAC,JPM,C,MSFT,GOOG,META,BTC-USD,ETH-USD
0,0.255201,0.212684,0.200000,0.199999,5.322189e-07,1.061141e-07,0.100000,6.804609e-08,0.200000,0.200000,8.498382e-08,5.329776e-07,0.099999
1,0.216310,0.196618,0.171802,0.014828,1.133697e-01,4.881012e-07,0.149999,1.413494e-07,0.200000,0.200000,4.999988e-02,9.249520e-02,0.007505
2,0.209147,0.187704,0.105063,0.019481,1.754582e-01,1.068138e-02,0.139319,5.701698e-06,0.199996,0.199998,4.999821e-02,9.958247e-02,0.000417
3,0.207480,0.184233,0.081376,0.021075,1.975491e-01,2.629624e-02,0.123704,2.016873e-07,0.200000,0.200000,4.999994e-02,9.999751e-02,0.000002
4,0.207109,0.183169,0.073924,0.026082,1.999944e-01,3.445978e-02,0.116314,2.778741e-06,0.199224,0.199999,4.999945e-02,9.998650e-02,0.000014


In [10]:
c = (
    alt.Chart(efficient_frontier)
    .mark_line(point=alt.OverlayMarkDef(color="#e60049", size=50), color="#e60049")
    .encode(
        x=alt.X(
            "volatility:Q", axis=alt.Axis(title="Volatility", format=".0%")
        ),
        y=alt.Y(
            "expected_return:Q",
            axis=alt.Axis(
                title="Expected Return",
                format=".0%",
            ),
            scale=alt.Scale(
                domain=[
                    efficient_frontier.loc[:, "expected_return"].min() * 0.99,
                    efficient_frontier.loc[:, "expected_return"].max() * 1.01,
                ]
            ),
        ),
        tooltip=[
            alt.Tooltip(f"{col}:Q", format=".2%")
            for col in asset_sectors.loc[:, "Asset"]
        ]
        + [
            alt.Tooltip("volatility:Q", format=".2%", title="Volatility"),
            alt.Tooltip(
                "expected_return:Q", format=".2%", title="Expected Return"
            ),
        ],
    )
).interactive().properties(width=400, height=300)

c


/Users/matiasmacazaga/python/portfolio_optimization_with_mosek/.venv/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [126]:
c.save("efficient_frontier.html")